# Split Data According to Label Rows

Split each `.flac` file according to it's `.csv`, i.e. create a feature for each row in the `.csv` with the corresponding label.
Label is `-1` if there was no whistle and `1` else.

In [14]:
import librosa
import numpy as np
np.set_printoptions(precision=4, suppress=True)
import os
import pandas as pd
from scipy.fftpack import fft, fftfreq
import soundfile as sf

sys.path.append(os.pardir)
from helper.load_data import load_data

In [2]:
import sys

# do not print warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter('ignore')

In [15]:
# determine path to `src/data`
cwd = os.getcwd()
src_dir = os.path.join(cwd, os.pardir)
data_dir = os.path.join(src_dir, 'data')

# load files
data_file_paths = load_data(data_dir)

In [16]:
feature_matrix = []
labels = []

max_iter = 3

for file in data_file_paths[:max_iter]:
    base = os.path.splitext(file)[0]
    extension = os.path.splitext(file)[1]
    
    if extension == '.flac' and os.path.isfile(base + '.csv'):
        label_file = pd.read_csv(base + '.csv')
        for index, row in label_file.iterrows():
            start = row['start']
            if start < 0: start = 0
            end = row['end']
            duration = end - start
            
            if duration <= 0:
                print(file)
                print(f'In row {index}')
                continue
            
            waveform, sample_rate = librosa.load(
                file,
                sr=None,    # do not resample file
                mono=True,
                offset=start,
                duration=duration)
            
            feature_matrix.append(fft(waveform))
            labels.append(-1 if (row['label'] == 'No_Whistle') else 1)

In [13]:
# print(labels[:20])
# print(feature_matrix[:20])